In [1]:
import csv
import pandas as pd
import numpy as np

# Read tsv file
r_body  = pd.read_csv('soc-redditHyperlinks-body.tsv', sep='\t')
r_title = pd.read_csv('soc-redditHyperlinks-title.tsv', sep='\t')


In [2]:
# print the first 5 rows of the dataframe
print(r_body.head())

# count all the multi edges
print(r_body.groupby(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']).size().reset_index(name='count').sort_values('count', ascending=False))

  SOURCE_SUBREDDIT TARGET_SUBREDDIT  POST_ID            TIMESTAMP  \
0  leagueoflegends  teamredditteams  1u4nrps  2013-12-31 16:39:58   
1       theredlion           soccer   1u4qkd  2013-12-31 18:18:37   
2     inlandempire           bikela  1u4qlzs  2014-01-01 14:54:35   
3              nfl              cfb  1u4sjvs  2013-12-31 17:37:55   
4       playmygame          gamedev  1u4w5ss  2014-01-01 02:51:13   

   LINK_SENTIMENT                                         PROPERTIES  
0               1  345.0,298.0,0.75652173913,0.0173913043478,0.08...  
1              -1  101.0,98.0,0.742574257426,0.019801980198,0.049...  
2               1  85.0,85.0,0.752941176471,0.0235294117647,0.082...  
3               1  1124.0,949.0,0.772241992883,0.0017793594306,0....  
4               1  715.0,622.0,0.777622377622,0.00699300699301,0....  
          SOURCE_SUBREDDIT TARGET_SUBREDDIT  count
128037  trendingsubreddits        changelog    548
114895       streetfighter              sf4    279
19991 

In [2]:
# show rows where link_sentiment is -1 (negative) and group by subreddit and count the number of rows
r = r_body[r_body['LINK_SENTIMENT'] == -1].groupby('SOURCE_SUBREDDIT')['LINK_SENTIMENT'].count()

# order by count
r = r.sort_values(ascending=False)
r

SOURCE_SUBREDDIT
subredditdrama         1437
circlebroke             702
drama                   518
shitliberalssay         407
circlejerkcopypasta     380
                       ... 
mathideas                 1
maturechat                1
mautitest                 1
mavericks                 1
10cloverfieldlane         1
Name: LINK_SENTIMENT, Length: 4154, dtype: int64

In [16]:
# show rows where link_sentiment is -1 (negative) and group by subreddit and count the number of rows
r = r_title[r_title['LINK_SENTIMENT'] == -1].groupby('SOURCE_SUBREDDIT')['LINK_SENTIMENT'].count()

# order by count
r = r.sort_values(ascending=False)
r


SOURCE_SUBREDDIT
subredditdrama     7550
bestof             3464
drama              2172
circlebroke2       1642
shitpost           1308
                   ... 
dogeawards            1
doge                  1
dogbootyattacks       1
nova                  1
lux                   1
Name: LINK_SENTIMENT, Length: 6217, dtype: int64